In [3]:
%pip install typing fastapi pypdf2 pymupdf tempfile os base64 traceback re collections math nltk faiss-cpu python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 3.10.0.0 Requires-Python >=2.7, !=3.0.*, !=3.1.*, !=3.2.*, !=3.3.*, <3.5; 3.7.4.2 Requires-Python >=2.7, !=3.0.*, !=3.1.*, !=3.2.*, !=3.3.*, <3.5
ERROR: Could not find a version that satisfies the requirement tempfile (from versions: none)
ERROR: No matching distribution found for tempfile


In [4]:
%pip uninstall -y fitz
%pip install PyMuPDF

  Using cached pymupdf-1.26.4-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.2 MB/s eta 0:00:00


In [5]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Alicia-RAG-Chatbot') # Ajusta esta ruta si es necesario

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# -*- coding: utf-8 -*-
"""mainipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1DNTN2NtuOrcD-eiQAl31Y937Wpt5   vqBJ
"""

# main.py
from typing import Any
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import fitz  # PyMuPDF
import tempfile
import os
import base64
import traceback
import re
from collections import defaultdict
import math
from nltk.tokenize import sent_tokenize





# Utilidades y limpieza de texto

In [7]:
def clean_pdf_text_robust(text):
    """Limpia texto de PDF de forma MÁS robusta para RAG, atacando patrones específicos."""
    if not text: return ""
    # --- PASOS DE LIMPIEZA GENERAL ---
    ligatures = {'ﬁ': 'fi', 'ﬂ': 'fl', 'ﬀ': 'ff', 'ﬃ': 'ffi', 'ﬄ': 'ffl'}
    for lig, repl in ligatures.items(): text = text.replace(lig, repl)
    text = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', text) # Unir palabras con guión
    text = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', text) # Segunda pasada
    text = re.sub(r'^\s*Página\s+\d+(\s+de\s+\d+)?\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Paginación
    text = re.sub(r'\b\d+\s*/\s*\d+\b', '', text) # Paginación X / Y
    text = re.sub(r'https?://[^\s/$.?#].[^\s]*', '', text, flags=re.IGNORECASE) # URLs http/https
    text = re.sub(r'\bwww\.[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b(?!\.)', '', text) # URLs www
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text) # Emails

    # --- REGLAS ESPECÍFICAS MEJORADAS ---
    text = re.sub(r'https?://opo\.cl/[a-zA-Z0-9]+', '', text, flags=re.IGNORECASE) # URLs opo.cl
    text = re.sub(r'\bopositatest\.com\b', '', text, flags=re.IGNORECASE) # Dominio específico
    text = re.sub(r'\bv\d+\.\d+\.\d+\b', '', text, flags=re.IGNORECASE) # Versión vX.Y.Z
    text = re.sub(r'/?\s*\+34\s*(\d{1,3}\s*){2,4}', '', text) # Teléfono +34
    text = re.sub(r'^\s*\d+\s+TEMARIO\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE)
    text = re.sub(r'^\s*Accede a los recursos.*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Línea recursos
    text = re.sub(r'^\s*Comprueba si tu temario.*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Línea actualizado
    text = re.sub(r'^\s*ORGANIZACIÓN DEL ESTADO\s*\|\s*TEMA\s*\d+\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Cabecera específica
    text = re.sub(r'^\s*RECURSOS\s*\n?\s*(GRÁFICOS)?\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Cabecera Recursos
    text = re.sub(r'^\s*\d+\s*$', '', text, flags=re.MULTILINE) # Líneas solo con número (experimental)
    # Eliminar bloque explicativo iconos (más agresivo)
    text = re.sub(r'^\s*RECURSOS\s+GRÁFICOS.*?simple vistazo\.', '', text, flags=re.IGNORECASE | re.DOTALL | re.MULTILINE)
    text = re.sub(r'^\s*PLAZOS\s+Sabemos que.*?simple vistazo\.', '', text, flags=re.IGNORECASE | re.DOTALL | re.MULTILINE)
    text = re.sub(r'^\s*(PLAZOS|Destacados|Pregunta de examen|Datos importantes|Negrita)\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Títulos sueltos iconos

    # --- PASOS DE NORMALIZACIÓN FINAL ---
    text = re.sub(r'[ \t\f\v]+', ' ', text) # Normalizar espacios horizontales
    text = re.sub(r' +\n', '\n', text) # Espacios antes de salto
    text = re.sub(r'\n +', '\n', text) # Espacios después de salto
    text = re.sub(r'\n{3,}', '\n\n', text) # Reducir saltos múltiples a 2
    text = re.sub(r'^\s*\n', '', text, flags=re.MULTILINE) # Eliminar líneas vacías residuales
    text = re.sub(r'^\s*[-•*o»·]\s+', '- ', text, flags=re.MULTILINE) # Normalizar viñetas
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text) # Caracteres de control
    text = text.strip() # Limpiar inicio/fin
    if text: text = text.rstrip('\n') + '\n\n' # Asegurar que termine con dos saltos
    return text

# Detección de portada y secciones matemáticas

In [8]:

import re

# --- DETECCIÓN PORTADA (MODIFICADA) ---
def is_likely_cover(page_text, page_number, num_total_pages):
    """Heurística para detectar portadas."""

    # 1. Limpieza y conteo inicial
    lines = [line for line in page_text.split('\n') if line.strip()]
    line_count = len(lines)
    text_length = len(page_text.strip())

    # 2. Regla heurística extendida (para las primeras 15 páginas)
    # Se considera portada/página preliminar si está en las primeras 15 páginas
    # y tiene muy poco contenido (pocas líneas o pocos caracteres).
    if page_number < 15 and (line_count < 15 or text_length < 200):
         return True

    # 3. Segunda regla heurística (para las primeras páginas, buscando palabras clave)
    # Esta regla es más estricta y busca contenido editorial específico.
    if page_number < 5 and line_count < 25:
        # He modificado el rango de búsqueda a 'page_number < 5' para concentrar
        # la búsqueda de palabras clave en las páginas iniciales, aunque podrías mantenerlo en 'page_number < 2'.

        # Patrón que busca palabras clave editoriales/legales
        if re.search(r'\b(temario|edición|editorial|reservados todos los derechos|oposici[oó]n|ISBN|Copyright)\b', page_text, re.IGNORECASE):
            return True

    # 4. Resultado por defecto
    return False



# Detección de bibliografía e imágenes

In [9]:

def detectar_paginas_resumen_biblio(pdf_path, max_paginas_finales_a_revisar=10):
    """
    Detecta páginas que contienen RESUMEN o BIBLIOGRAFÍA por separado.
    """
    paginas_resumen = []
    paginas_biblio = []

    resumen_keywords = ['RESUMEN', 'CONCLUSIÓ']
    biblio_keywords = ['BIBLIOGRAFÍA', 'REFERENCIAS', 'WEBGRAFÍA']

    try:
        doc = fitz.open(pdf_path)
        num_total_pages = len(doc)
        start_page_index = max(0, num_total_pages - max_paginas_finales_a_revisar)

        for page_num in range(start_page_index, num_total_pages):
            page = doc.load_page(page_num)
            text = page.get_text("text").upper()

            if not text or text.isspace():
                continue

            # Detectar resumen
            if any(re.search(r'(?:^[ \t]*|\n[ \t]*)' + kw + r'\b', text) for kw in resumen_keywords):
                paginas_resumen.append(page_num)

            # Detectar bibliografía
            if any(re.search(r'(?:^[ \t]*|\n[ \t]*)' + kw + r'\b', text) for kw in biblio_keywords):
                paginas_biblio.append(page_num)

        doc.close()

    except Exception as e:
        print(f"WARN (detectar_resumen_biblio): Error procesando {pdf_path}: {e}")

    return paginas_resumen, paginas_biblio


def detect_image_regions_on_page(
    page: Any,
    merge_close_distance: int = 5,
    min_area: int = 1000,
    detect_drawings: bool = False,
    debug: bool = False
) -> list:
    """
    Detecta regiones probables de imágenes (y opcionalmente dibujos vectoriales)
    en una página de PyMuPDF, retornando bounding boxes fusionadas y filtradas.

    Args:
        page (fitz.Page): Página de PyMuPDF sobre la que se detectan imágenes.
        merge_close_distance (int): Distancia máxima (en puntos) para fusionar
            bounding boxes que se solapan o están muy cerca.
        min_area (int): Área mínima (en puntos^2) para no descartar regiones pequeñas.
        detect_drawings (bool): Si True, intentará detectar regiones vectoriales
            (get_drawings()) y tratarlas como imágenes.
        debug (bool): Si True, muestra mensajes de debug.

    Returns:
        list[dict]: Lista de regiones detectadas, cada una con:
            {
              "bbox": (x0, y0, x1, y1),
              "type": "image" | "drawing"
            }
    """
    all_regions = []
    try:
        # --------------------------------------------------------
        # 1. DETECCIÓN DE IMÁGENES BITMAP
        # --------------------------------------------------------
        images_info = page.get_images(full=True)
        for img_info in images_info:
            xref = img_info[0]
            if xref == 0:
                continue  # ignorar imágenes inline o inválidas
            try:
                # Obtener los rectángulos donde se dibuja esta imagen (puede haber varios)
                img_rects = page.get_image_rects(xref)
                for rect in img_rects:
                    bbox = rect.irect  # (x0, y0, x1, y1) con coords enteras
                    x0, y0, x1, y1 = bbox
                    area = (x1 - x0) * (y1 - y0)
                    if area >= min_area:
                        all_regions.append({"bbox": bbox, "type": "image"})
                    elif debug:
                        print(f"DEBUG: Descartando imagen muy pequeña bbox={bbox}, area={area}")
            except Exception as err_rects:
                if debug:
                    print(f"DEBUG: No se pudo obtener rects de imagen xref={xref}: {err_rects}")

        # --------------------------------------------------------
        # 2. DETECCIÓN DE "DRAWINGS" VECTORIALES (OPCIONAL)
        # --------------------------------------------------------
        if detect_drawings:
            try:
                drawings = page.get_drawings()
                for d in drawings:
                    # 'type' puede ser: 'l' (line), 're' (rectangle),
                    # 'f' (fill?), 'cs' (curves?), etc.
                    # Ajusta según tus necesidades de filtrado.
                    # Aquí descartamos líneas simples:
                    if d['type'] == 'l':
                        continue
                    bbox = d['rect'].irect
                    x0, y0, x1, y1 = bbox
                    area = (x1 - x0) * (y1 - y0)
                    if area >= min_area:
                        all_regions.append({"bbox": bbox, "type": "drawing"})
                    elif debug:
                        print(f"DEBUG: Descartando dibujo pequeño bbox={bbox}, area={area}")
            except Exception as err_draw:
                if debug:
                    print(f"DEBUG: Error detectando dibujos vectoriales: {err_draw}")

        # --------------------------------------------------------
        # 3. FUSIÓN DE BBOXES CERCANOS O SOLAPADOS
        # --------------------------------------------------------
        merged_regions = _merge_bounding_boxes(all_regions, merge_close_distance, debug=debug)

        if debug:
            print(f"DEBUG: detect_image_regions_on_page => {len(all_regions)} sin fusionar, {len(merged_regions)} tras fusión")

        return merged_regions

    except Exception as e:
        print(f"WARN: Error detectando imágenes/dibujos en página: {e}")
        return []

def _merge_bounding_boxes(regions: list, close_dist: int, debug: bool = False) -> list:
    """
    Funde bounding boxes que se solapan o están muy cerca, retornando una
    nueva lista de regiones. Cada región es un dict con:
      { "bbox": (x0, y0, x1, y1), "type": "image" / "drawing" }.

    - close_dist: se considerará 'cerca' si la distancia entre 2 rects
      es menor o igual a close_dist.
    - Este método hace un loop iterativo hasta que no haya merges nuevos.

    Return: lista de dicts con bboxes fusionadas.
    """

    # Para fusionar rects, necesitamos una pequeña función de "check solape" y "unión"
    def rects_are_close_or_overlap(r1, r2, threshold):
        """Retorna True si r1 y r2 se solapan o la distancia entre ellos es <= threshold."""
        (x0a, y0a, x1a, y1a) = r1
        (x0b, y0b, x1b, y1b) = r2

        # 1) Si se solapan en x e y (overlap check)
        overlap_x = not (x1a < x0b or x1b < x0a)
        overlap_y = not (y1a < y0b or y1b < y0a)
        if overlap_x and overlap_y:
            return True

        # 2) Si no solapan, calculamos distancia mínima entre los rects
        #    Si es <= threshold, consideramos "cerca".
        dist = _min_dist_between_rects(r1, r2)
        return dist <= threshold

    def merge_rects(r1, r2):
        """Devuelve el bounding box que cubre ambos rects."""
        (x0a, y0a, x1a, y1a) = r1
        (x0b, y0b, x1b, y1b) = r2
        return (
            min(x0a, x0b),
            min(y0a, y0b),
            max(x1a, x1b),
            max(y1a, y1b)
        )

    changed = True
    while changed:
        changed = False
        merged_list = []
        skip_indices = set()
        n = len(regions)

        for i in range(n):
            if i in skip_indices:
                continue
            r1 = regions[i]
            merged = False
            for j in range(i+1, n):
                if j in skip_indices:
                    continue
                r2 = regions[j]

                if r1["type"] == r2["type"] or True:
                    # Si quisieras mantener separado "image" vs "drawing", podrías
                    # fusionar solo si r1["type"] == r2["type"]. O fusionar siempre.
                    if rects_are_close_or_overlap(r1["bbox"], r2["bbox"], close_dist):
                        # Merge them
                        new_bbox = merge_rects(r1["bbox"], r2["bbox"])
                        # Podríamos unificar el 'type'; aquí escogemos la del primero
                        # o creamos algo como "mixed"
                        new_type = r1["type"] if r1["type"] == r2["type"] else "mixed"
                        merged_list.append({"bbox": new_bbox, "type": new_type})
                        skip_indices.add(j)
                        merged = True
                        if debug:
                            print(f"DEBUG: Merged {r1['bbox']} + {r2['bbox']} => {new_bbox}")
                        break
            if not merged:
                # No fusionamos r1 con nadie
                merged_list.append(r1)

        if len(merged_list) < len(regions):
            # Hubo fusión => repetimos
            regions = merged_list
            changed = True
        else:
            # Sin cambio => terminamos
            regions = merged_list

    return regions


from math import sqrt

def _min_dist_between_rects(r1, r2):
    """
    Calcula la distancia mínima entre dos rects (x0, y0, x1, y1)
    si no se solapan.
    """
    x0a, y0a, x1a, y1a = r1
    x0b, y0b, x1b, y1b = r2

    # Si se solapan en x, la distancia en x es 0; de lo contrario,
    # es la diferencia entre los bordes más cercanos.
    if x1a < x0b:
        dx = x0b - x1a
    elif x1b < x0a:
        dx = x0a - x1b
    else:
        dx = 0

    # Lo mismo para y.
    if y1a < y0b:
        dy = y0b - y1a
    elif y1b < y0a:
        dy = y0a - y1b
    else:
        dy = 0

    # Distancia euclidiana
    return sqrt(dx*dx + dy*dy)


print("INFO: Funciones auxiliares STEM definidas (Fórmulas, Encabezados, Imágenes).")

INFO: Funciones auxiliares STEM definidas (Fórmulas, Encabezados, Imágenes).


# Detección de índice


In [10]:


def detectar_paginas_indice(pdf_path, max_paginas_a_revisar=None, umbral_min_lineas=5):
    """
    Intenta detectar las páginas del índice (Tabla de Contenido) en un PDF.

    Utiliza heurísticas mejoradas basadas en patrones de texto comunes,
    combinando numeración jerárquica, palabras clave y opcionalmente
    la presencia de números de página al final de la línea.

    Args:
        pdf_path (str): Ruta al archivo PDF.
        max_paginas_a_revisar (int): Número máximo de páginas iniciales a revisar.
        umbral_min_lineas (int): Mínimo de líneas de texto requeridas en una página
                                 para siquiera considerarla como índice.

    Returns:
        list: Una lista de índices de página (basados en 0) que probablemente
              contienen el índice. Lista vacía si no se detecta ninguno o hay error.
    """
    paginas_indice_detectadas = []
    if not os.path.exists(pdf_path):
         print(f"ERROR: (detectar_paginas_indice) Archivo no encontrado: {pdf_path}")
         return paginas_indice_detectadas

    doc = None
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"WARN: (detectar_paginas_indice) Error al abrir PDF '{pdf_path}': {e}")
        return paginas_indice_detectadas

    # --- Heurísticas ---
    # Regex para numeración como 1., 1.1, 1.1.1., CAPÍTULO 1, TEMA 2, etc. (más flexible)
    patron_numeracion_jerarquica = re.compile(
        r"^\s*([0-9]+(\.[0-9]+)*\.?\s+|"  # 1., 1.1, 1.1.
        r"(CAP[IÍ]TULO|TEMA|SECCI[OÓ]N|PARTE)\s+[0-9IVXLCDM]+\b\.?\s*).*",
        re.IGNORECASE
    )
    # Regex para palabras clave comunes en índices/sumarios (usando search)
    patron_palabras_clave = re.compile(
        r"^\s*(INTRODUCCI[OÓ]N|PR[OÓ]LOGO|CONCLUSI[OÓ]N|EP[IÍ]LOGO|BIBLIOGRAF[IÍ]A|WEBGRAF[IÍ]A|REFERENCIAS|RESUMEN|[IÍ]NDICE|CONTENIDO|SUMARIO|ANEXO|GLOSARIO)\b",
        re.IGNORECASE
    )
    # Regex para líneas que probablemente terminan en un número de página (puede estar precedido por puntos o espacios)
    patron_linea_con_pagina = re.compile(r".*[.\s]\s*(\d+)\s*$")

    num_paginas_a_escanear = min(max_paginas_a_revisar, doc.page_count)
    posible_indice_activo = False # Flag para detectar índices multi-página

    print(f"INFO: Escaneando hasta {num_paginas_a_escanear} páginas para índice en '{os.path.basename(pdf_path)}'")

    for num_pagina in range(num_paginas_a_escanear):
        try:
            pagina = doc.load_page(num_pagina)
            # Usar bloques puede ser un poco más robusto para la separación de líneas
            bloques = pagina.get_text("blocks")
            lineas = []
            for b in bloques:
                # b[4] contiene el texto del bloque, puede tener \n internos
                block_text = b[4]
                # Dividir por nueva línea y limpiar
                lineas.extend(line.strip() for line in block_text.split('\n') if line.strip())

            num_total_lineas = len(lineas)

            # Ignorar páginas casi vacías o portadas detectadas
            if num_total_lineas < umbral_min_lineas or is_likely_cover("\n".join(lineas), num_pagina, doc.page_count):
                # print(f"DEBUG P{num_pagina+1}: Ignorada (líneas={num_total_lineas} < {umbral_min_lineas} or portada)")
                posible_indice_activo = False # Si no es índice, rompe la cadena
                continue

            contador_lineas_patron = 0
            contador_palabras_clave = 0
            contador_lineas_con_pagina = 0

            for linea in lineas:
                if patron_numeracion_jerarquica.match(linea):
                    contador_lineas_patron += 1
                # Usamos search para palabras clave, más flexible a indentación
                if patron_palabras_clave.search(linea):
                    contador_palabras_clave += 1
                if patron_linea_con_pagina.match(linea):
                    # Verificación adicional: asegurarse de que el número no sea parte de la numeración inicial
                    match_num_inicial = patron_numeracion_jerarquica.match(linea)
                    num_final_match = patron_linea_con_pagina.match(linea)
                    if num_final_match:
                         num_final_str = num_final_match.group(1)
                         # Evitar contar si el número final es el mismo que el inicial (p.ej., "1. Título 1")
                         if not (match_num_inicial and linea.strip().endswith(num_final_str) and len(linea.split()) < 4):
                              contador_lineas_con_pagina += 1


            ratio_lineas_patron = contador_lineas_patron / num_total_lineas
            ratio_lineas_con_pagina = contador_lineas_con_pagina / num_total_lineas

            # --- Lógica de Decisión Mejorada ---
            es_pagina_indice = False
            score = 0.0

            # Puntuación base por estructura de numeración (alta importancia)
            score += ratio_lineas_patron * 0.6

            # Puntuación por líneas terminando en número (media importancia)
            score += ratio_lineas_con_pagina * 0.3

            # Bonus por presencia de palabras clave (menor importancia individual, pero ayuda)
            if contador_palabras_clave > 0:
                score += 0.1 # Bonus fijo pequeño si hay al menos una
            if contador_palabras_clave > 2:
                score += 0.1 # Bonus adicional si hay varias

            # Umbral base para considerar índice
            umbral_score_base = 0.25 # Ajustar según sea necesario

            # Umbral más bajo si la página anterior fue índice (continuación)
            umbral_score_continuacion = 0.18

            if posible_indice_activo:
                if score >= umbral_score_continuacion:
                    es_pagina_indice = True
            else:
                 if score >= umbral_score_base:
                    es_pagina_indice = True

            # Refinamiento: Una página con muchas palabras clave pero CERO estructura podría ser un falso positivo
            # O una página con ALTA estructura pero pocas líneas podría no serlo.
            # (La comprobación de umbral_min_lineas ya ayuda con lo segundo)
            if contador_palabras_clave > 1 and contador_lineas_patron == 0 and contador_lineas_con_pagina == 0:
                 # Si SOLO tiene palabras clave y ninguna otra estructura, probablemente no sea índice (podría ser intro/conclusión)
                 # A menos que tenga MUCHAS líneas con palabras clave? Podría ser un índice simple.
                 if num_total_lineas > 10 and (contador_palabras_clave / num_total_lineas > 0.3): # Si >30% de lineas son keywords
                     pass # Probablemente un índice simple basado en keywords, mantener es_pagina_indice si score fue suficiente
                 else:
                     es_pagina_indice = False # Descartar si no cumple la condición anterior


            # DEBUGGING INTERNO
            print(f"  Pág {num_pagina + 1}: Lines={num_total_lineas}, "
                  f"RatioPatron={ratio_lineas_patron:.2f} ({contador_lineas_patron}), "
                  f"Keywords={contador_palabras_clave}, "
                  f"RatioPgNum={ratio_lineas_con_pagina:.2f} ({contador_lineas_con_pagina}), "
                  f"Score={score:.3f} -> Índice? {es_pagina_indice} (ActivoPrev? {posible_indice_activo})")

            if es_pagina_indice:
                if num_pagina not in paginas_indice_detectadas:
                     paginas_indice_detectadas.append(num_pagina)
                posible_indice_activo = True
            else:
                # Si la página no cumple, se rompe la posible cadena de índice
                posible_indice_activo = False

        except Exception as e:
            print(f"WARN: (detectar_paginas_indice) Error procesando página {num_pagina} del PDF: {e}")
            posible_indice_activo = False # Resetear en caso de error
            continue

    if doc:
        doc.close()

    # Post-procesamiento: a veces puede detectar una página suelta entre otras.
    # Si tenemos [0, 2], pero no 1, es menos probable que 2 sea índice.
    # Podríamos requerir bloques contiguos, pero por simplicidad lo dejamos así por ahora.

    print(f"INFO: Páginas de índice detectadas: {[p+1 for p in paginas_indice_detectadas]}") # Mostrar páginas base 1
    return paginas_indice_detectadas



# Extracción y limpieza principal

In [11]:

def extract_and_clean_pdf_smart(pdf_path,
                                     use_ocr_threshold=50,
                                     language='spa', # 'language' no se usa directamente aquí, quizás en OCR
                                     max_index_pages_to_scan=15,
                                     max_summary_biblio_pages_to_scan=10,
                                     debug_prints=False): # Añadido parámetro debug_prints
    """
    Extrae texto de PDF, detecta fórmulas/imágenes,
    omite portadas/índices, limpia texto, y elimina solo el bloque de
    bibliografía sin eliminar páginas enteras.

    - Aplica detect_formulas_in_text() para ver si hay LaTeX literal.
    - Marca si el texto parece contener expresiones matemáticas (heurística).
    - Detecta secciones matemáticas (teorema, definición, demostración, etc.)
      a nivel global.
    """
    all_formulas_detected = []
    all_image_regions = {}
    valid_pages_text = []
    omitted_pages_info = []
    # debug_prints = False # Se recibe como parámetro ahora

    try:
        if not os.path.exists(pdf_path):
            print(f"Error GRAVE: No se encontró el archivo PDF: {pdf_path}")
            # Devolver None o un dict vacío con error es mejor que solo None
            return {"error": f"File not found: {pdf_path}", "cleaned_text": "", "detected_formulas": [], "detected_image_regions": {}, "omitted_pages": [], "heuristic_math_detected": False, "detected_math_sections": {}}


        pdf_basename = os.path.basename(pdf_path)

        # --- Pre-detección de ÍNDICE ---
        if debug_prints: print(f"DEBUG ({pdf_basename}): Pre-detectando páginas de índice (hasta {max_index_pages_to_scan} págs)...")
        paginas_indice_detectadas = detectar_paginas_indice(
            pdf_path, max_paginas_a_revisar=max_index_pages_to_scan
        )
        if paginas_indice_detectadas:
            if debug_prints: print(f"DEBUG ({pdf_basename}): Posibles páginas de ÍNDICE (0-based): {paginas_indice_detectadas}")

        # --- Pre-detección de RESUMEN/BIBLIO ---
        if debug_prints: print(f"DEBUG ({pdf_basename}): Pre-detectando págs Resumen/Biblio (últimas {max_summary_biblio_pages_to_scan})...")
        paginas_resumen_detectadas, paginas_biblio_detectadas = detectar_paginas_resumen_biblio(
            pdf_path, max_paginas_finales_a_revisar=max_summary_biblio_pages_to_scan
        )
        if paginas_resumen_detectadas:
            if debug_prints: print(f"DEBUG ({pdf_basename}): Páginas con posible RESUMEN (0-based): {paginas_resumen_detectadas}")
        if paginas_biblio_detectadas:
            if debug_prints: print(f"DEBUG ({pdf_basename}): Páginas con posible BIBLIOGRAFÍA (0-based): {paginas_biblio_detectadas}")

        # --- Crear conjunto de páginas a omitir solo para índice ---
        # Convertir a 0-based si las funciones de detección devuelven 1-based
        # Asumiendo que devuelven 0-based:
        paginas_a_omitir_previamente = set(paginas_indice_detectadas)

        doc = fitz.open(pdf_path)
        num_total_pages = len(doc)
        if debug_prints: print(f"DEBUG ({pdf_basename}): Procesando {num_total_pages} páginas (Modo STEM).")

        for page_num in range(num_total_pages):
            page_num_real = page_num + 1 # Para logs y referencias (1-based)
            if debug_prints: print(f"DEBUG ({pdf_basename}): Procesando pág {page_num_real}/{num_total_pages}...")

            # --- Omisión de páginas índice ---
            if page_num in paginas_a_omitir_previamente:
                reason = "Índice (pre-detectado)"
                omitted_pages_info.append((page_num_real, reason))
                if debug_prints:
                    print(f"  -> OMITIDA ({reason}).")
                continue

            # --- Extracción básica de texto ---
            page_raw_text = ""
            page_raw_text_strip = ""
            try:
                page = doc.load_page(page_num) # Cargar página dentro del bucle
                page_raw_text = page.get_text("text", sort=True)
                page_raw_text_strip = page_raw_text.strip() if page_raw_text else ""
                if debug_prints and not page_raw_text_strip:
                    print(f"  -> WARN: get_text devolvió vacío o solo espacios.")
            except Exception as getTextErr:
                print(f"WARN ({pdf_basename}): get_text falló pág {page_num_real}: {getTextErr}.")
                omitted_pages_info.append((page_num_real, f"Error get_text: {getTextErr}"))
                continue # Saltar página si falla la extracción básica

            # --- Lógica de OCR (Opcional, si se requiere) ---
            used_ocr = False
            if not page_raw_text_strip or len(page_raw_text_strip) < use_ocr_threshold:
                # Aquí iría la llamada a una función OCR si decides implementarla
                # page_raw_text_ocr = apply_ocr_to_page(page, language=language)
                # if page_raw_text_ocr and len(page_raw_text_ocr.strip()) > len(page_raw_text_strip):
                #     page_raw_text = page_raw_text_ocr
                #     page_raw_text_strip = page_raw_text.strip()
                #     used_ocr = True
                #     if debug_prints: print(f"  -> INFO: OCR aplicado (resultado > {use_ocr_threshold} chars).")
                # elif debug_prints:
                #     print(f"  -> INFO: Texto < {use_ocr_threshold} chars, OCR no aplicado o sin mejora.")
                pass # Placeholder para OCR

            # --- Comprobación de texto vacío (Post-OCR si aplica) ---
            if not page_raw_text_strip:
                reason = "Sin texto válido (post-OCR)" if used_ocr else "Sin texto válido"
                omitted_pages_info.append((page_num_real, reason))
                if debug_prints: print(f"  -> OMITIDA ({reason}).")
                continue

            # --- Heurística de portada ---
            if is_likely_cover(page_raw_text_strip, page_num, num_total_pages):
                reason = "Portada (heurística)"
                omitted_pages_info.append((page_num_real, reason))
                if debug_prints: print(f"  -> OMITIDA ({reason}).")
                continue

            # --- Cortar bibliografía si corresponde (Solo si la página fue pre-detectada) ---
            final_page_text = page_raw_text_strip # Usar texto strip para búsqueda
            original_length_before_bib_cut = len(final_page_text)
            bibliography_cut_applied = False

            if page_num in paginas_biblio_detectadas:
                # Buscar keywords en mayúsculas para robustez, pero cortar el original
                biblio_keywords_regex = r'^\s*(BIBLIOGRAFÍA|REFERENCIAS|WEBGRAFÍA)\s*$' # Más específico, inicio de línea
                # Intentar buscar desde el final de la página hacia atrás podría ser más robusto
                lines = final_page_text.splitlines()
                cut_index = -1
                for i in range(len(lines) - 1, -1, -1):
                     if re.search(biblio_keywords_regex, lines[i].strip().upper()):
                         # Encontrar la posición de inicio de esta línea en el texto original
                         try:
                            cut_index = final_page_text.rindex(lines[i])
                            break
                         except ValueError:
                            pass # Seguir buscando si la línea no se encuentra exactamente

                if cut_index != -1:
                    final_page_text = final_page_text[:cut_index].strip()
                    bibliography_cut_applied = True
                    if debug_prints:
                        print(f"  -> INFO: Texto cortado por keyword de bibliografía encontrada.")

                    # Si tras cortar no queda texto, omitir la página
                    if not final_page_text:
                        reason = "Texto eliminado por contenido de bibliografía"
                        omitted_pages_info.append((page_num_real, reason))
                        if debug_prints: print(f"  -> OMITIDA ({reason}).")
                        continue


            # Detectar imágenes (asumiendo que la función existe)
            page_image_bboxes = detect_image_regions_on_page(page)
            if page_image_bboxes:
                all_image_regions[page_num_real] = page_image_bboxes
                if debug_prints: print(f"  -> INFO: Detectadas {len(page_image_bboxes)} regiones de imagen.")

            # --- Página Aceptada (añadir texto final) ---
            valid_pages_text.append(final_page_text)
            if debug_prints: print(f"  -> ACEPTADA (len: {len(final_page_text)} chars).")

        # --- Fin del bucle de páginas ---
        doc.close()

        # --- Resumen de omisiones ---
        print("\n" + "-"*20 + f" Resumen Omisiones ({pdf_basename}) " + "-"*20)
        if not omitted_pages_info:
            print("INFO: No se omitió ninguna página.")
        else:
            omitted_by_reason = defaultdict(list)
            for page, reason in omitted_pages_info:
                omitted_by_reason[reason].append(page)
            print(f"INFO: Omitidas {len(omitted_pages_info)}/{num_total_pages} páginas:")
            for reason, pages in sorted(omitted_by_reason.items()):
                pages.sort()
                # Agrupar páginas consecutivas para mejor lectura
                grouped_pages = []
                if pages:
                    start_range = pages[0]
                    end_range = pages[0]
                    for i in range(1, len(pages)):
                        if pages[i] == end_range + 1:
                            end_range = pages[i]
                        else:
                            if start_range == end_range:
                                grouped_pages.append(str(start_range))
                            else:
                                grouped_pages.append(f"{start_range}-{end_range}")
                            start_range = end_range = pages[i]
                    # Añadir el último rango/página
                    if start_range == end_range:
                        grouped_pages.append(str(start_range))
                    else:
                        grouped_pages.append(f"{start_range}-{end_range}")
                print(f"  - Razón: '{reason}', Páginas: {', '.join(grouped_pages)}")
        print("-"*(42 + len(f" Resumen Omisiones ({pdf_basename}) ")))

        # --- Limpieza final del texto concatenado ---
        if not valid_pages_text:
            print(f"ERROR ({pdf_basename}): No se aceptó ninguna página válida.")
            return {
                "cleaned_text": "",
                "detected_formulas": [],
                "detected_image_regions": {},
                "omitted_pages": omitted_pages_info,
                "heuristic_math_detected": False,
                "detected_math_sections": {}
            }

        full_raw_text = "\n\n".join(valid_pages_text) # Unir páginas aceptadas
        if debug_prints: print(f"DEBUG ({pdf_basename}): {len(valid_pages_text)} págs aceptadas. Limpiando texto concatenado...")

        # Asumiendo que clean_pdf_text_robust está definido.
        cleaned_text = clean_pdf_text_robust(full_raw_text)
        if debug_prints: print(f"DEBUG ({pdf_basename}): Limpieza completada. Longitud final: {len(cleaned_text)} chars.")

        if not cleaned_text or cleaned_text.isspace():
            print(f"WARN ({pdf_basename}): Texto final limpio vacío.")
            # Devolver el estado aunque el texto esté vacío
            return {
                "cleaned_text": "",
                "detected_formulas": all_formulas_detected,
                "detected_image_regions": all_image_regions,
                "omitted_pages": omitted_pages_info,
                "heuristic_math_detected": False, # No hay texto para analizar
                "detected_math_sections": {}
            }



        # --- Devolver Resultados ---
        return {
            "cleaned_text": cleaned_text,
            "detected_formulas": all_formulas_detected,       # LaTeX literal
            "detected_image_regions": all_image_regions,
            "omitted_pages": omitted_pages_info,

        }

    except FileNotFoundError:
 # Ser más específico con la excepción
        print(f"Error GRAVE: No se encontró el PDF: {pdf_path}")
        return {"error": f"File not found: {pdf_path}", "cleaned_text": "", "detected_formulas": [], "detected_image_regions": {}, "omitted_pages": [], "heuristic_math_detected": False, "detected_math_sections": {}}
    except Exception as e:
        print(f"Error GRAVE procesando PDF {os.path.basename(pdf_path)} ({type(e).__name__}): {e}")
        traceback.print_exc()
        return {"error": f"Processing error: {e}", "cleaned_text": "", "detected_formulas": [], "detected_image_regions": {}, "omitted_pages": [], "heuristic_math_detected": False, "detected_math_sections": {}}


In [13]:
# --- Código de extracción y limpieza ---
pdf_path = "/content/119-2014-02-19-Carroll.AliciaEnElPaisDeLasMaravillas (1).pdf"
resultado = extract_and_clean_pdf_smart(pdf_path=pdf_path, debug_prints=True) # Desactiva prints internos si son muchos
resultado

DEBUG (119-2014-02-19-Carroll.AliciaEnElPaisDeLasMaravillas (1).pdf): Pre-detectando páginas de índice (hasta 15 págs)...
INFO: Escaneando hasta 15 páginas para índice en '119-2014-02-19-Carroll.AliciaEnElPaisDeLasMaravillas (1).pdf'
  Pág 5: Lines=23, RatioPatron=0.00 (0), Keywords=1, RatioPgNum=0.09 (2), Score=0.126 -> Índice? False (ActivoPrev? False)
  Pág 6: Lines=24, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 7: Lines=34, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 8: Lines=35, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 9: Lines=35, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 10: Lines=35, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 11: Lines=35, RatioPa

{'cleaned_text': 'I. EN LA MADRIGUERA DEL CONEJO\nAlicia empezaba ya a cansarse de estar sentada con\nsu hermana a la orilla del río, sin tener nada que\nhacer: había echado un par de ojeadas al libro que su\nhermana estaba leyendo, pero no tenía dibujos ni\ndiálogos. «¿Y de qué sirve un libro sin dibujos ni diálogos?», se preguntaba Alicia.\nAsí pues, estaba pensando (y pensar le costaba\ncierto esfuerzo, porque el calor del día la había dejado\nsoñolienta y atontada) si el placer de tejer una guirnalda de margaritas la compensaría del trabajo de levantarse y coger las margaritas, cuando de pronto\nsaltó cerca de ella un Conejo Blanco de ojos rosados.\nNo había nada muy extraordinario en esto, ni\ntampoco le pareció a Alicia\nmuy extraño oír que el conejo se decía a sí mismo:\n«¡Dios mío! ¡Dios mío! ¡Voy\na llegar tarde!» (Cuando\npensó en ello después, decidió que, desde luego, hubiera debido sorprenderla mucho, pero en aquel momento le pareció lo más natural del mundo). Pero\ncuando

In [14]:
import re
from nltk.tokenize import sent_tokenize
import nltk

# Asegúrate de tener el tokenizador 'punkt'
# nltk.download('punkt')

# La función de chunking sigue siendo la misma. La incluimos aquí para que el código esté completo.
def split_into_chunks_with_metadata(text, metadata, max_chunk_size=256, overlap_size=50):
    if not isinstance(text, str):
        raise ValueError("El texto de entrada debe ser un string.")
    if overlap_size >= max_chunk_size:
        raise ValueError("El tamaño del solapamiento debe ser menor que el tamaño máximo del chunk.")
    sentences = sent_tokenize(text, language='spanish')
    chunks = []
    current_chunk_words = []
    for sentence in sentences:
        sentence_words = sentence.split()
        if len(current_chunk_words) + len(sentence_words) > max_chunk_size and current_chunk_words:
            chunk_text = " ".join(current_chunk_words)
            chunks.append({'text': chunk_text, 'metadata': metadata})
            current_chunk_words = current_chunk_words[-overlap_size:]
        current_chunk_words.extend(sentence_words)
    if current_chunk_words:
        chunk_text = " ".join(current_chunk_words)
        chunks.append({'text': chunk_text, 'metadata': metadata})
    return chunks

# --- NUEVA VERSIÓN DEL ORQUESTADOR ---
def process_text_hierarchically(text_content, source_name):
    """
    Procesa un texto usando una expresión regular estricta que solo coincide
    con títulos de capítulo reales.
    """
    print("Iniciando procesamiento con patrón regex definitivo...")

    # El patrón que solo acepta títulos en mayúsculas
    chapter_pattern = re.compile(
        r'^\s*([IVXLCDM\d]+)\.\s+([A-ZÁÉÍÓÚÜÑ\d\s:;.,\'"-]+)\s*$',
        re.MULTILINE
    )

    # Con un patrón fiable, re.split es la herramienta más limpia.
    # El resultado será: [intro, num1, titulo1, contenido1, num2, titulo2, contenido2, ...]
    parts = chapter_pattern.split(text_content)

    all_chunks = []

    # El primer elemento siempre es el texto de introducción
    intro_text = parts[0].strip()
    if intro_text:
        intro_metadata = {'source': source_name, 'chapter': 'Introducción'}
        all_chunks.extend(split_into_chunks_with_metadata(intro_text, intro_metadata))

    # Procesamos el resto de las partes, que vienen en grupos de 3:
    # (número, título, contenido).
    for i in range(1, len(parts), 3):
        chapter_number = parts[i].strip().replace('.', '') # Limpia puntos extra
        chapter_title = parts[i+1].strip()
        chapter_content = parts[i+2].strip()

        if not chapter_content: continue

        full_chapter_name = f"Capítulo {chapter_number}: {chapter_title}"
        print(f"  -> Capítulo detectado correctamente: {full_chapter_name}")

        chapter_metadata = {'source': source_name, 'chapter': full_chapter_name}
        all_chunks.extend(split_into_chunks_with_metadata(chapter_content, chapter_metadata))

    return all_chunks

In [15]:
import nltk
nltk.download('punkt_tab')

def process_pdf_to_structured_chunks(pdf_path):
    """
    Flujo de trabajo completo: extrae, hace chunking jerárquico y enriquece metadatos.
    """
    print("--- INICIO DEL PROCESO ---")

    # 1. EXTRACCIÓN Y LIMPIEZA
    print("\n[Paso 1] Extrayendo y limpiando texto del PDF...")
    resultado = extract_and_clean_pdf_smart(pdf_path=pdf_path)
    texto_limpio = resultado["cleaned_text"]
    topic = os.path.basename(pdf_path)
    print(f"Texto extraído. Longitud: {len(texto_limpio)} caracteres.")

    # 2. CHUNKING ESTRUCTURAL (JERÁRQUICO)
    print("\n[Paso 2] Realizando chunking jerárquico por capítulos...")
    structured_chunks = process_text_hierarchically(texto_limpio, source_name= 'Alicia en el pais de las maravllas')
    print(f"Chunking estructural completado. Se generaron {len(structured_chunks)} chunks iniciales.")

    # 3. ENRIQUECIMIENTO DE METADATOS
    print("\n[Paso 3] Enriqueciendo cada chunk con metadatos de contenido...")
    final_chunks_with_metadata = []
    for i, chunk_data in enumerate(structured_chunks):
        chunk_text = chunk_data['text']

        # Obtenemos los metadatos estructurales ya existentes
        metadata = chunk_data['metadata']



        # El diccionario 'chunk_data' ahora tiene el texto y los metadatos combinados
        final_chunks_with_metadata.append(chunk_data)

    print("Enriquecimiento completado.")
    print("\n--- PROCESO FINALIZADO ---")
    return final_chunks_with_metadata


# ==============================================================================
# EJECUCIÓN
# ==============================================================================

if __name__ == "__main__":
    # Usa la ruta a tu PDF real aquí
    pdf_path = "/content/119-2014-02-19-Carroll.AliciaEnElPaisDeLasMaravillas (1).pdf"

    chunks = process_pdf_to_structured_chunks(pdf_path)

    print(f"\nTotal de chunks finales generados: {len(chunks)}\n")

    # Muestra los metadatos de cada chunk para verificar
    print("=== METADATOS FINALES POR CHUNK ===")
    for i, chunk_meta in enumerate(chunks):
        print(f"\n--- Chunk {i+1} ---")
        print(chunk_meta)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


--- INICIO DEL PROCESO ---

[Paso 1] Extrayendo y limpiando texto del PDF...
INFO: Escaneando hasta 15 páginas para índice en '119-2014-02-19-Carroll.AliciaEnElPaisDeLasMaravillas (1).pdf'
  Pág 5: Lines=23, RatioPatron=0.00 (0), Keywords=1, RatioPgNum=0.09 (2), Score=0.126 -> Índice? False (ActivoPrev? False)
  Pág 6: Lines=24, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 7: Lines=34, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 8: Lines=35, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 9: Lines=35, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 10: Lines=35, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.00 (0), Score=0.000 -> Índice? False (ActivoPrev? False)
  Pág 11: Lines=35, RatioPatron=0.00 (0), Keywords=1, RatioPgNum=0.00 (0

In [16]:


# --- Celda de Carga de Claves y Cliente OpenAI (Reemplaza tu bloque antiguo con este) ---

from openai import OpenAI
from dotenv import load_dotenv
import os

# 1. Cargar las variables de entorno desde el archivo .env
load_dotenv()

# 2. Obtener la clave de API desde las variables de entorno
#    Usamos os.getenv() para leer la variable que definimos en el archivo .env
openai_api_key = os.getenv("OPENAI_API_KEY")

# 3. (MUY IMPORTANTE) Verificación de seguridad y usabilidad
if not openai_api_key:
    # Si la clave no se encuentra, detenemos la ejecución con un error claro.
    raise ValueError("ERROR: La clave de API de OpenAI no se encontró. "
                     "Asegúrate de crear un archivo '.env' en el mismo directorio que este notebook "
                     "y añadir la línea: OPENAI_API_KEY='sk-...'")
else:
    print("✅ Clave de API de OpenAI cargada exitosamente desde el archivo .env.")

# 4. Inicializa el cliente usando la clave cargada de forma segura
client = OpenAI(
    api_key=openai_api_key
)

print("✅ Cliente de OpenAI inicializado.")

# --- Tu bloque de prueba (sin cambios, sigue siendo útil) ---
try:
    print("\nRealizando una llamada de prueba a la API de embeddings...")
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=["Este es un texto de prueba para generar un embedding."]
    )
    embedding_vector = response.data[0].embedding
    print("✅ Embedding generado exitosamente.")
    print(f"   El embedding tiene {len(embedding_vector)} dimensiones.")

except Exception as e:
    print(f"❌ Ocurrió un error al contactar la API de OpenAI: {e}")

✅ Clave de API de OpenAI cargada exitosamente desde el archivo .env.
✅ Cliente de OpenAI inicializado.

Realizando una llamada de prueba a la API de embeddings...
✅ Embedding generado exitosamente.
   El embedding tiene 1536 dimensiones.


In [17]:
%pip install faiss-cpu langchain_openai rank_bm25 langchain_google_genai sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
go

In [18]:
  # Install the faiss library
import faiss
import numpy as np


# --- FUNCIÓN PARA GENERAR EMBEDDINGS ---

def generate_embeddings_for_chunks(chunks_with_metadata, batch_size=50, model="text-embedding-3-small"):
    """
    Genera embeddings para una lista de chunks de texto usando la API de OpenAI en lotes.

    Args:
        chunks_with_metadata (list[dict]): La lista de chunks, donde cada elemento
                                           es un diccionario {'text': ..., 'metadata': ...}.
        batch_size (int): El número de textos a procesar en cada llamada a la API.
        model (str): El modelo de embedding de OpenAI a utilizar.

    Returns:
        list[dict]: La misma lista de chunks, pero ahora cada diccionario también
                    contiene una clave 'embedding' con su vector numérico.
    """
    chunks_with_embeddings = []

    # Iteramos sobre la lista de chunks en lotes del tamaño de 'batch_size'
    for i in range(0, len(chunks_with_metadata), batch_size):
        # 1. Selecciona el lote actual de chunks
        current_batch = chunks_with_metadata[i:i + batch_size]

        # 2. Extrae solo el texto de cada chunk en el lote
        texts_to_embed = [chunk['text'] for chunk in current_batch]

        print(f"Procesando lote {i//batch_size + 1}/{(len(chunks_with_metadata) - 1)//batch_size + 1}... "
              f"({len(texts_to_embed)} textos)")

        try:
            # 3. Llama a la API de OpenAI con el lote de textos
            response = client.embeddings.create(
                model=model,
                input=texts_to_embed
            )

            # 4. Extrae los embeddings de la respuesta
            embeddings = [item.embedding for item in response.data]

            # 5. Asigna cada embedding a su chunk correspondiente
            for j, chunk in enumerate(current_batch):
                chunk['embedding'] = embeddings[j] # Añade la nueva clave 'embedding'
                chunks_with_embeddings.append(chunk)

            # Pausa opcional para no exceder los límites de la API (rate limits)
            time.sleep(1) # Pausa de 1 segundo entre lotes

        except Exception as e:
            print(f"Error procesando el lote que empieza en el índice {i}: {e}")
            # Opcional: podrías decidir saltar este lote y continuar, o detener el proceso
            continue

    return chunks_with_embeddings


# --- EJEMPLO DE USO ---

if __name__ == "__main__":
    # 1. Simula tu lista de chunks ya procesada (usa solo unos pocos para el ejemplo)
    # En tu código real, usarías la lista completa de 143 chunks.
    sample_chunks_processed = [
        {'text': 'Alicia empezaba ya a cansarse de estar sentada con su hermana a la orilla del río, sin tener nada que hacer...',
         'metadata': {'source': 'Alicia.pdf', 'chapter': 'Capítulo I: EN LA MADRIGUERA DEL CONEJO'}},
        {'text': '...cuando de pronto saltó cerca de ella un Conejo Blanco de ojos rosados. No había nada muy extraordinario en esto...',
         'metadata': {'source': 'Alicia.pdf', 'chapter': 'Capítulo I: EN LA MADRIGUERA DEL CONEJO'}},
        {'text': '—¡Curiorífico y curiorífico! —exclamó Alicia. ¡Ahora me estoy estirando como el telescopio más largo que haya existido jamás!',
         'metadata': {'source': 'Alicia.pdf', 'chapter': 'Capítulo II: EL CHARCO DE LÁGRIMAS'}},
        # ... y así sucesivamente para todos tus chunks
    ]

    print(f"Se van a procesar {len(sample_chunks_processed)} chunks de ejemplo.")

    # 2. Llama a la función para generar los embeddings
    # (Usa un batch_size pequeño para este ejemplo)
    chunks_final_data = generate_embeddings_for_chunks(chunks, batch_size=2)

    # 3. Verifica el resultado
    print("\n--- PROCESO DE EMBEDDING COMPLETADO ---")
    if chunks_final_data:
        print(f"Se generaron embeddings para {len(chunks_final_data)} chunks.")

        # Imprime el primer chunk para ver la nueva estructura
        print("\nEjemplo del primer chunk con su embedding:")
        first_chunk = chunks_final_data[0]

        # Usamos pprint para una mejor visualización
        print({
            'text': first_chunk['text'][:50] + '...', # Muestra solo el inicio del texto
            'metadata': first_chunk['metadata'],
            'embedding': f"[Vector de {len(first_chunk['embedding'])} dimensiones]" # Muestra un resumen del embedding
        })
    else:
        print("No se generaron embeddings.")

Se van a procesar 3 chunks de ejemplo.
Procesando lote 1/72... (2 textos)
Error procesando el lote que empieza en el índice 0: name 'time' is not defined
Procesando lote 2/72... (2 textos)
Error procesando el lote que empieza en el índice 2: name 'time' is not defined
Procesando lote 3/72... (2 textos)
Error procesando el lote que empieza en el índice 4: name 'time' is not defined
Procesando lote 4/72... (2 textos)
Error procesando el lote que empieza en el índice 6: name 'time' is not defined
Procesando lote 5/72... (2 textos)
Error procesando el lote que empieza en el índice 8: name 'time' is not defined
Procesando lote 6/72... (2 textos)
Error procesando el lote que empieza en el índice 10: name 'time' is not defined
Procesando lote 7/72... (2 textos)
Error procesando el lote que empieza en el índice 12: name 'time' is not defined
Procesando lote 8/72... (2 textos)
Error procesando el lote que empieza en el índice 14: name 'time' is not defined
Procesando lote 9/72... (2 textos)
Err

In [19]:
# --- Añade esto al final de tu script de generación de embeddings ---

import numpy as np
import faiss
import pickle
import os

# Suponiendo que 'chunks_final_data' es tu lista de chunks con texto, metadatos y embeddings

# --- CONFIGURACIÓN DE NOMBRES DE ARCHIVO ---
# Usemos nombres que correspondan a nuestro nuevo libro
INDEX_PATH = "alicia.index"
TEXTS_PATH = "alicia_texts.pkl"
METAS_PATH = "alicia_metas.pkl"

# 1. Separar los datos para guardarlos
embeddings = np.array([chunk['embedding'] for chunk in chunks_final_data], dtype=np.float32)
texts = [chunk['text'] for chunk in chunks_final_data]
metadatas = [chunk['metadata'] for chunk in chunks_final_data]

print(f"Datos separados: {len(embeddings)} embeddings, {len(texts)} textos, {len(metadatas)} metadatos.")
print(f"Dimensiones del vector de embedding: {embeddings.shape[1]}")

# 2. Crear y entrenar el índice FAISS
# Usamos un índice simple 'IndexFlatL2' que es bueno para empezar
d = embeddings.shape[1]  # Dimensión de los vectores
index = faiss.IndexFlatL2(d)
print(f"Índice FAISS vacío creado con dimensión {d}.")

# Añadir los vectores al índice
index.add(embeddings)
print(f"Se han añadido {index.ntotal} vectores al índice FAISS.")

# 3. Guardar todo en archivos
print(f"Guardando índice FAISS en '{INDEX_PATH}'...")
faiss.write_index(index, INDEX_PATH)

print(f"Guardando textos en '{TEXTS_PATH}'...")
with open(TEXTS_PATH, 'wb') as f:
    pickle.dump(texts, f)

print(f"Guardando metadatos en '{METAS_PATH}'...")
with open(METAS_PATH, 'wb') as f:
    pickle.dump(metadatas, f)

print("\n--- ¡Proceso de guardado completado! ---")
print("Ahora puedes usar estos 3 archivos en tu script de búsqueda RAG.")

Datos separados: 143 embeddings, 143 textos, 143 metadatos.
Dimensiones del vector de embedding: 1536
Índice FAISS vacío creado con dimensión 1536.
Se han añadido 143 vectores al índice FAISS.
Guardando índice FAISS en 'alicia.index'...
Guardando textos en 'alicia_texts.pkl'...
Guardando metadatos en 'alicia_metas.pkl'...

--- ¡Proceso de guardado completado! ---
Ahora puedes usar estos 3 archivos en tu script de búsqueda RAG.


In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Alicia-RAG-Chatbot') # Ajusta esta ruta si es necesario

Mounted at /content/drive


In [20]:
# Ejemplo para cargar claves en un notebook

from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not OPENAI_API_KEY or not GOOGLE_API_KEY:
    print("⚠️ ADVERTENCIA: No se encontraron las claves de API en el entorno.")
    print("Asegúrate de tener un archivo .env o de haber configurado los secretos de Colab.")
else:
    print("✅ Claves de API cargadas exitosamente.")

✅ Claves de API cargadas exitosamente.


In [21]:
# --- Celda 2: Parámetros de Configuración y Variables Globales ---

# --- Parámetros de Archivos y Azure ---
INDEX_PATH = "alicia.index"
TEXTS_PATH = "alicia_texts.pkl"
METAS_PATH = "alicia_metas.pkl"
AZURE_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-3-small" # Asegúrate que coincide con tu deployment
AZURE_API_VERSION = "2024-02-01" # O la versión que uses ej: "2023-05-15"

# --- Parámetros para Búsqueda Híbrida y Reranking ---
K_FAISS_INITIAL = 100  # Número de candidatos a recuperar de FAISS
K_BM25_INITIAL = 100   # Número de candidatos a recuperar de BM25
K_RERANK = 80         # Número de candidatos a pasar al reranker (<= K_FAISS + K_BM25)
K_FINAL = 3
USE_DYNAMIC_K = True        # True para usar K dinámico, False para usar K_FINAL fijo
RERANKER_SCORE_THRESHOLD = 1.5 # Umbral mínimo para considerar un chunk (ajustar según scores observados)
MIN_CHUNKS_DYNAMIC = 3      # Mínimo de chunks a devolver si USE_DYNAMIC_K es True
MAX_CHUNKS_DYNAMIC = 7      # Máximo de chunks a devolver si USE_DYNAMIC_K es True         # Número final de chunks a devolver al LLM            # Número final de chunks a devolver al LLM
RERANKER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-12-v2' # Modelo CrossEncoder

# --- Variables Globales para inicialización única (se llenarán en la primera ejecución) ---
is_retriever_initialized = False
# Objetos principales:
embeddings_model = None
faiss_index = None
texts = None
metadatas = None
bm25 = None
reranker = None
# Opcional: stop words en español si usas NLTK
# spanish_stopwords = stopwords.words('spanish')

print("INFO: Celda 2 - Parámetros y Variables Globales definidas.")
print(f"  - K_FAISS_INITIAL: {K_FAISS_INITIAL}")
print(f"  - K_BM25_INITIAL: {K_BM25_INITIAL}")
print(f"  - K_RERANK: {K_RERANK}")
print(f"  - K_FINAL: {K_FINAL}")
print(f"  - RERANKER_MODEL: {RERANKER_MODEL}")
print("--- Fin Celda 2 ---")

INFO: Celda 2 - Parámetros y Variables Globales definidas.
  - K_FAISS_INITIAL: 100
  - K_BM25_INITIAL: 100
  - K_RERANK: 80
  - K_FINAL: 3
  - RERANKER_MODEL: cross-encoder/ms-marco-MiniLM-L-12-v2
--- Fin Celda 2 ---


In [22]:
# --- Celda 3: Funciones Auxiliares ---

def simple_tokenizer(text):
    """Tokenizador simple: minúsculas y split por espacios."""
    if not isinstance(text, str):
        return []
    return text.lower().split()

# Opcional: Tokenizador más robusto con NLTK (requiere descargas en Celda 1)
# def nltk_tokenizer(text):
#     """Tokenizador con NLTK: minúsculas, palabras, sin puntuación ni stopwords."""
#     if not isinstance(text, str):
#         return []
#     words = word_tokenize(text.lower(), language='spanish')
#     # Asegúrate que spanish_stopwords está definida si descomentas esto
#     # return [word for word in words if word.isalnum() and word not in spanish_stopwords]
#     return [word for word in words if word.isalnum()] # Sin stopwords

# Elige tu tokenizador preferido aquí (¡asegúrate que la función existe!)
tokenizer_for_bm25 = simple_tokenizer
# tokenizer_for_bm25 = nltk_tokenizer # Si prefieres NLTK


def norm_score(score, min_val, max_val):
    """
    Normaliza un score a un rango [0, 1].
    Maneja el caso donde min_val == max_val para evitar división por cero.
    """
    if min_val == max_val:
        # Si todos los scores son iguales, podemos devolver 0.5 (neutral) o 1 si el score es ese valor, o 0.
        # Devolver 0 si min_val == max_val y score == min_val (o cualquier score ya que son todos iguales)
        # o 0.5 para indicar que no hay varianza. Elegiremos 0.5 como un valor neutral.
        # Otra opción es devolver 1.0 si solo hay un resultado y es positivo, o 0.0 si es 0.
        # O, si solo hay un elemento, su score normalizado puede ser 1.
        return 1.0 if score > 0 else 0.0 # Si hay un solo score y es > 0, es el "mejor"
    if max_val - min_val == 0: # Otra forma de chequear división por cero
        return 0.5 # O 1.0 si el score es el único valor
    return (score - min_val) / (max_val - min_val)

import re

def calcular_pesos_dinamicos(query: str, subject: str = None) -> tuple[float, float]:
    """
    Analiza la query educativa y el tema (opcional) y ajusta pesos entre BM25 y Embeddings.
    Devuelve (peso_bm25, peso_emb).
    """
    query_lower = query.lower()
    query_original = query # Para checks de mayúsculas

    # --- Pesos Base ---
    peso_bm25 = 0.4
    peso_emb = 0.6
    razon_principal = "Default (ligero sesgo Embedding)"
    detalles_razon = []

    # --- 1. Indicadores de ALTA ESPECIFICIDAD (Prioridad Alta para BM25) ---

    # 1.1. Citas exactas (texto entre comillas)
    if re.search(r'"[^"]+"', query_original): # Busca texto entre comillas dobles
        peso_bm25 = 0.85
        peso_emb = 0.15
        razon_principal = "Cita Exacta"
        detalles_razon.append("BM25 priorizado para coincidencia literal.")
        print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
        return peso_bm25, peso_emb

    # 1.bis. Definición de Término Clave Específico (Ej: "elipsis", "hipérbaton")
    definicion_keywords_specific_term = [
        "define", "definición de", "definir", "significa",
        "qué es", "que es", "cuál es el significado de",
        "concepto de"
    ]
    term_to_define_specific = ""
    for keyword in definicion_keywords_specific_term:
        # Patrón para "keyword X" o "keyword 'X'" o "keyword "X""
        # o para "X keyword" (menos común para estas keywords pero podría pasar)
        # Priorizamos "keyword X"
        if query_lower.startswith(keyword + " "):
            potential_term = query_lower[len(keyword)+1:].strip()
            # Quitar comillas y signos de interrogación del término
            potential_term = re.sub(r"['\"?¿!¡]$", "", potential_term).strip()
            potential_term = re.sub(r"^['\"]", "", potential_term).strip()

            # Si la query original tenía el término entre comillas, es buena señal
            if f"'{potential_term}'" in query_original or f'"{potential_term}"' in query_original:
                 term_to_define_specific = potential_term
                 break
            # Si no, tomarlo si es corto
            elif len(potential_term.split()) <= 3:
                 term_to_define_specific = potential_term
                 break

    if term_to_define_specific and len(term_to_define_specific.split()) <= 3 and len(query.split()) < 8 : # Término corto, query no demasiado larga
        # Evitar que una pregunta conceptual larga que casualmente empieza con "qué es la vida..." caiga aquí
        # Si la query es más larga, es probable que sea más conceptual.
        peso_bm25 = 0.80 # Alta prioridad para BM25 para encontrar el término exacto
        peso_emb = 0.20
        razon_principal = "Definición de Término Clave Específico"
        detalles_razon.append(f"Término detectado: '{term_to_define_specific}'. BM25 fuertemente priorizado.")
        print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
        return peso_bm25, peso_emb


    # 1.2. Búsqueda de Leyes, Artículos, Teoremas específicos
    if re.search(r'\b(ley|artículo|teorema|postulado|axioma|principio)\s+([0-9]+|[xviíclmd]+|[A-Za-z\s]+)\b', query_lower, re.IGNORECASE):
        peso_bm25 = 0.75
        peso_emb = 0.25
        razon_principal = "Ley/Artículo/Teorema Específico"
        detalles_razon.append("BM25 priorizado para identificadores exactos.")
        print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
        return peso_bm25, peso_emb

    # 1.3. Fórmulas o Ecuaciones
    if re.search(r'\b[a-zA-Z]\s*=\s*[a-zA-Z0-9]|\b[a-zA-Z]\w*\([a-zA-Z\d,\s]*\)|[a-zA-Z]\w*_[a-zA-Z\d]|\w\^[2-9]\b', query_original):
        if subject in ["Física", "Biología", "Matemáticas", "Química"]: # Más probable que sea una fórmula
            peso_bm25 = 0.70
            peso_emb = 0.30
            razon_principal = "Posible Fórmula/Ecuación"
            detalles_razon.append(f"BM25 priorizado en {subject} para coincidencia estructural.")
            print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
            return peso_bm25, peso_emb

    # --- 2. Indicadores de ESPECIFICIDAD MEDIA (Favorecen BM25, pero con espacio para semántica) ---

    # 2.1. Nombres Propios
    nombres_propios_candidatos = re.findall(r'\b[A-ZÁÉÍÓÚÑ][a-záéíóúñ]{2,}(?:\s+[A-ZÁÉÍÓÚÑ][a-záéíóúñ]{1,})*\b', query_original)
    if nombres_propios_candidatos:
        if not (len(nombres_propios_candidatos) == 1 and query_original.startswith(nombres_propios_candidatos[0]) and len(query.split()) > 3):
            peso_bm25 = max(peso_bm25, 0.65) # Aumenta si el default era menor, o lo establece
            peso_emb = 1.0 - peso_bm25
            if razon_principal.startswith("Default"): razon_principal = "Nombre Propio Detectado"
            detalles_razon.append(f"Candidatos NP: {nombres_propios_candidatos}. BM25 priorizado.")

    # 2.2. Fechas, Años, Siglos
    if re.search(r'\b\d{3,4}\b', query_lower) or \
       re.search(r'\bsiglo\s+(?:[xviíclmd]+|[0-9]+)\b', query_lower) or \
       re.search(r'\b(año|fecha)\s+\d{1,4}\b', query_lower) or \
       re.search(r'\b\d{1,2}(?:/| de |-| del )\w+(?:/| de |-| del )\d{2,4}\b', query_lower):
        peso_bm25 = max(peso_bm25, 0.70)
        peso_emb = 1.0 - peso_bm25
        if razon_principal.startswith("Default") or "Nombre Propio" in razon_principal: razon_principal = "Fecha/Año/Siglo Detectado"
        detalles_razon.append("BM25 priorizado para especificidad temporal.")
        if subject == "Historia":
            peso_bm25 = max(peso_bm25, 0.75) # Aún más para Historia
            peso_emb = 1.0 - peso_bm25
            detalles_razon.append("Alta prioridad BM25 en Historia.")

    # 2.3. Acrónimos y Términos Técnicos Muy Específicos
    acronimos_candidatos = re.findall(r'\b[A-ZÁÉÍÓÚÑ]{2,}\b', query_original)
    if acronimos_candidatos and not query_original.isupper():
        if not (len(acronimos_candidatos) == 1 and query_original.startswith(acronimos_candidatos[0])):
            peso_bm25 = max(peso_bm25, 0.60)
            peso_emb = 1.0 - peso_bm25
            if razon_principal.startswith("Default") or "Nombre Propio" in razon_principal or "Fecha" in razon_principal:
                razon_principal = "Acrónimo/Término Técnico Específico Detectado"
            detalles_razon.append(f"Candidatos Acrónimo: {acronimos_candidatos}. BM25 con peso incrementado.")


    # --- 3. Indicadores de BÚSQUEDA DE DEFINICIONES (Equilibrio, si no es ya muy específico) ---
    # Esta regla se aplica si las de ALTA ESPECIFICIDAD (incluida 1.bis) no se activaron y retornaron.
    definicion_keywords_general = ["define", "definición de", "definir", "significa", "concepto de"]
    que_es_keywords_general = ["qué es", "que es", "cual es el significado de", "cuál es el significado de"]

    is_general_definition_request = False
    if any(keyword in query_lower for keyword in definicion_keywords_general) or \
       any(query_lower.startswith(keyword) for keyword in que_es_keywords_general):
        is_general_definition_request = True

    if is_general_definition_request:
        # Si ya se marcó como muy específico (nombre propio, fecha, acrónimo), mantenemos BM25 alto,
        # pero si la razón principal aún es "Default" o algo menos específico.
        if peso_bm25 < 0.6: # Solo ajusta si no es ya específico por reglas anteriores
            peso_bm25 = 0.55
            peso_emb = 0.45
            razon_principal = "Petición de Definición General"
            detalles_razon.append("Pesos ligeramente inclinados a BM25 para literalidad, pero con semántica.")
        else:
            detalles_razon.append("Petición de definición, pero query ya tenía especificidad media/alta.")


    # --- 4. Indicadores de CONCEPTUALIDAD (Prioridad para Embeddings) ---
    concept_keywords_strong = ["explica", "describe el proceso de", "analiza las causas de", "compara y contrasta",
                               "cuál es la importancia de", "interpreta", "relación entre", "impacto de",
                               "evolución de", "fundamentos de", "teoría de"]
    concept_keywords_medium = ["cómo funciona", "por qué ocurre", "cuáles son las características",
                               "tipos de", "función de", "origen de", "propiedades de"]

    is_conceptual = False
    conceptual_keyword_found = ""
    for keyword in concept_keywords_strong:
        if keyword in query_lower:
            is_conceptual = True
            conceptual_keyword_found = keyword
            detalles_razon.append(f"Palabra clave conceptual fuerte detectada: '{keyword}'.")
            break
    if not is_conceptual:
        for keyword in concept_keywords_medium:
            if keyword in query_lower:
                is_conceptual = True
                conceptual_keyword_found = keyword
                detalles_razon.append(f"Palabra clave conceptual media detectada: '{keyword}'.")
                break

    if is_conceptual:
        # Si es una pregunta conceptual sobre un término muy específico (ya capturado por NP, Fecha, Acrónimo)
        # Ej: "Explica el impacto de la Peste Negra" -> Peste Negra (NP) + Explica (Conceptual)
        if peso_bm25 >= 0.65 : # Ya era muy específico
            peso_bm25 = 0.55 # Mantenemos algo de BM25 para el término, pero damos espacio a la explicación
            peso_emb = 0.45
            razon_principal = "Pregunta Conceptual Muy Específica"
            detalles_razon.append(f"Término específico combinado con petición conceptual ('{conceptual_keyword_found}').")
        elif peso_bm25 >= 0.55 and peso_bm25 < 0.65: # Especificidad media
            peso_bm25 = 0.40
            peso_emb = 0.60
            razon_principal = "Pregunta Conceptual con Especificidad Media"
            detalles_razon.append(f"Término con especificidad media combinado con petición conceptual ('{conceptual_keyword_found}').")
        else: # Pregunta conceptual más general
            peso_bm25 = 0.25
            peso_emb = 0.75
            razon_principal = "Pregunta Conceptual General"
            detalles_razon.append(f"Mayor peso para Embeddings debido a '{conceptual_keyword_found}'.")


    # --- 5. Ajustes por Asignatura (si se proporciona y no hay una regla fuerte dominante) ---
    if subject and (razon_principal.startswith("Default") or "Petición de Definición General" in razon_principal):
        original_razon_principal = razon_principal # Guardar por si no se modifica
        if subject == "Lengua Castellana":
            if "analiza el poema" in query_lower or "figuras retóricas" in query_lower or "estilo de" in query_lower or "comentario de texto" in query_lower:
                peso_bm25 = 0.3
                peso_emb = 0.7
                razon_principal = f"Conceptual (Lengua - Análisis Literario)"
            elif "regla gramatical" in query_lower or "ortografía de" in query_lower or "sintaxis de" in query_lower:
                peso_bm25 = 0.6
                peso_emb = 0.4
                razon_principal = f"Específico (Lengua - Gramática/Ortografía)"
        elif subject == "Historia":
            if "batalla de" in query_lower or "tratado de" in query_lower or "reinado de" in query_lower or "guerra de" in query_lower:
                if peso_bm25 < 0.65: # Solo si no fue ya capturado por NP/Fecha con alta prioridad
                    peso_bm25 = 0.65
                    peso_emb = 0.35
                    razon_principal = f"Evento Específico (Historia)"

        if original_razon_principal != razon_principal: # Si se aplicó una regla de asignatura
             detalles_razon.append(f"Ajuste por asignatura '{subject}'.")


    # --- 6. Ajuste final por longitud de la query (si aún es default o poco definido) ---
    # Se aplica si ninguna regla fuerte o de especificidad media/conceptual clara dominó
    if razon_principal.startswith("Default") or \
       ("Petición de Definición General" in razon_principal and peso_bm25 == 0.55) or \
       (peso_bm25 >= 0.35 and peso_bm25 <= 0.45 and not is_conceptual): # Default o ligeramente inclinado a Emb sin ser conceptual fuerte

        num_words_query = len(query.split())
        if num_words_query > 10:
            peso_bm25 = 0.30
            peso_emb = 0.70
            razon_principal = "Ajuste por Longitud (Larga -> Conceptual)"
            detalles_razon.append(f"Query larga ({num_words_query} palabras), favoreciendo semántica.")
        elif num_words_query < 4:
            peso_bm25 = 0.50 # Si era default (0.4), lo sube un poco para términos cortos
            peso_emb = 0.50
            razon_principal = "Ajuste por Longitud (Corta -> Equilibrio/Específica)"
            detalles_razon.append(f"Query corta ({num_words_query} palabras), buscando equilibrio o término.")


    print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
    return peso_bm25, peso_emb

def print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb):
    """Función auxiliar para imprimir la información de los pesos."""
    print(f"  INFO DinamicWeights: Razón Principal = {razon_principal}")
    if detalles_razon:
        for detalle in detalles_razon:
            print(f"    - {detalle}")
    print(f"  INFO DinamicWeights: Pesos Asignados -> BM25={peso_bm25:.2f}, Embedding={peso_emb:.2f}")



print("INFO: Celda 3 - Funciones auxiliares definidas (tokenizer, pesos, normalización).")
print(f"  - Usando tokenizer: {tokenizer_for_bm25.__name__}")
print("--- Fin Celda 3 ---")

INFO: Celda 3 - Funciones auxiliares definidas (tokenizer, pesos, normalización).
  - Usando tokenizer: simple_tokenizer
--- Fin Celda 3 ---


In [23]:
topic = 'Alicia en el pais de las maravillas'

In [24]:

# Asumo que las importaciones necesarias como numpy, faiss, pickle, etc., ya están en tu archivo.
# Asegúrate de importar la clase correcta:
from rank_bm25 import BM25Okapi
from langchain_openai import OpenAIEmbeddings # <--- CAMBIO: Importar esta clase
from sentence_transformers import CrossEncoder
# ... (resto de tus importaciones y variables globales como is_retriever_initialized)

# Asumo que las importaciones y variables globales ya están definidas antes de esta función.
# Librerías necesarias:
# import faiss, pickle, os, traceback
# import numpy as np
# from rank_bm25 import BM25Okapi
# from langchain_openai import OpenAIEmbeddings
# from sentence_transformers import CrossEncoder

def my_hybrid_rerank_retriever(query: str) -> str:
    """
    Función retriever completa que usa búsqueda híbrida (FAISS + BM25), fusión de scores,
    reranking con CrossEncoder y devuelve el contexto final como un string.
    Carga todos los recursos necesarios en la primera llamada.
    """
    # Las variables globales se acceden y modifican aquí
    global is_retriever_initialized, embeddings_model, faiss_index, texts, metadatas, bm25, reranker

    # --- Bloque de Inicialización (se ejecuta solo la primera vez) ---
    if not is_retriever_initialized:
        print("INFO: Inicializando el retriever HÍBRIDO por primera vez...")
        try:
            # --- SECCIÓN CORREGIDA ---
            # 1. Cargar modelo de Embedding de OpenAI (CON INDENTACIÓN CORRECTA)
            print("  Inicializando: 1. Cargando modelo Embedding de OpenAI...")

            # LangChain buscará automáticamente la variable de entorno "OPENAI_API_KEY"
            # que ya hemos cargado con load_dotenv().
            if not os.getenv("OPENAI_API_KEY"):
                raise ValueError("ERROR: La variable de entorno OPENAI_API_KEY no está definida.")
            else:
                print("     Variable de entorno OPENAI_API_KEY encontrada.")

            embedding_model_name = "text-embedding-3-small"
            embeddings_model = OpenAIEmbeddings(model=embedding_model_name)
            print(f"     Modelo Embedding OpenAI ({embedding_model_name}) cargado.")
            # --- FIN DE LA SECCIÓN CORREGIDA ---


            # 2. Cargar índice FAISS
            print("  Inicializando: 2. Cargando índice FAISS...")
            if not os.path.exists(INDEX_PATH):
                 raise FileNotFoundError(f"No se encontró el archivo de índice FAISS en: {INDEX_PATH}")
            faiss_index = faiss.read_index(INDEX_PATH)
            print(f"     Índice FAISS cargado desde '{INDEX_PATH}' ({faiss_index.ntotal} vectores).")

            # 3. Cargar textos y metadatos
            print("  Inicializando: 3. Cargando textos y metadatos...")
            if not os.path.exists(TEXTS_PATH): raise FileNotFoundError(f"Archivo no encontrado: {TEXTS_PATH}")
            if not os.path.exists(METAS_PATH): raise FileNotFoundError(f"Archivo no encontrado: {METAS_PATH}")
            with open(TEXTS_PATH, "rb") as f:
                texts = pickle.load(f)
            with open(METAS_PATH, "rb") as f:
                metadatas = pickle.load(f)
            print(f"     Textos ({len(texts)}) y Metadatos ({len(metadatas)}) cargados.")

            # 4. Verificación Crítica de Tamaños
            print("  Inicializando: 4. Verificando tamaños...")
            if not (faiss_index.ntotal == len(texts) == len(metadatas)):
                error_msg = f"¡ERROR CRÍTICO DE TAMAÑO! FAISS={faiss_index.ntotal}, Textos={len(texts)}, Metadatos={len(metadatas)}."
                print(error_msg)
                raise ValueError(error_msg)
            else:
                print("     OK: Tamaños coinciden.")

            # 5. Inicializar BM25
            print(f"  Inicializando: 5. Tokenizando documentos para BM25 ({tokenizer_for_bm25.__name__})...")
            if not isinstance(texts, list) or not all(isinstance(t, str) for t in texts):
                 raise TypeError("La variable 'texts' debe ser una lista de strings para BM25.")
            tokenized_docs = [tokenizer_for_bm25(txt) for txt in texts]
            bm25 = BM25Okapi(tokenized_docs)
            print("     Índice BM25 creado.")

            # 6. Inicializar Reranker (CrossEncoder)
            print(f"  Inicializando: 6. Cargando modelo Reranker '{RERANKER_MODEL}'...")
            reranker = CrossEncoder(RERANKER_MODEL)
            print("     Reranker cargado.")

            # 7. Marcar como inicializado
            is_retriever_initialized = True
            print("INFO: Inicialización del retriever HÍBRIDO completada.")

        except Exception as e:
            print(f"ERROR FATAL inicializando el retriever híbrido: {e}")
            traceback.print_exc()
            raise RuntimeError("Fallo al inicializar el retriever híbrido.") from e
    # --- Fin Bloque de Inicialización ---

    # --- Bloque de Búsqueda Híbrida y Reranking ---
    print(f"\n--- (RAG Híbrido + Rerank) Buscando contexto para: '{query}' ---")
    if not is_retriever_initialized:
        raise RuntimeError("El retriever no está inicializado. Hubo un error previo.")

    try:
        # 1. Obtener embedding de la consulta
        print("  1. Obteniendo embedding de OpenAI...")
        query_embedding = embeddings_model.embed_query(query)
        query_embedding_np = np.array([query_embedding], dtype=np.float32)
        print("     Embedding obtenido.")

        # 2. Búsqueda FAISS (vectorial)
        print(f"  2. Realizando búsqueda FAISS (k={K_FAISS_INITIAL})...")
        distances, faiss_indices = faiss_index.search(query_embedding_np, K_FAISS_INITIAL)
        faiss_sims = 1.0 / (1.0 + distances[0])
        faiss_results = {idx: sim for idx, sim in zip(faiss_indices[0], faiss_sims) if idx != -1}
        print(f"     Búsqueda FAISS -> {len(faiss_results)} candidatos.")

        # 3. Búsqueda BM25 (palabras clave)
        print(f"  3. Realizando búsqueda BM25 (k={K_BM25_INITIAL})...")
        tokenized_query = tokenizer_for_bm25(query)
        all_bm25_scores = bm25.get_scores(tokenized_query)
        bm25_top_indices = np.argsort(all_bm25_scores)[::-1][:K_BM25_INITIAL]
        bm25_results = {idx: all_bm25_scores[idx] for idx in bm25_top_indices if all_bm25_scores[idx] > 0}
        print(f"     Búsqueda BM25 -> {len(bm25_results)} candidatos.")

        # 4. Fusión Híbrida con Pesos Dinámicos
        print("  4. Fusionando resultados...")
        peso_bm25, peso_emb = calcular_pesos_dinamicos(query, topic)
        candidate_ids = set(faiss_results.keys()) | set(bm25_results.keys())
        print(f"     Total IDs candidatos únicos: {len(candidate_ids)}")

        faiss_scores_list = list(faiss_results.values())
        min_faiss, max_faiss = (min(faiss_scores_list), max(faiss_scores_list)) if faiss_scores_list else (0.0, 0.0)
        bm25_scores_list = list(bm25_results.values())
        min_bm25, max_bm25 = (min(bm25_scores_list), max(bm25_scores_list)) if bm25_scores_list else (0.0, 0.0)

        hybrid_scores = {}
        for idx in candidate_ids:
            score_f = faiss_results.get(idx, 0.0)
            score_b = bm25_results.get(idx, 0.0)
            norm_f = norm_score(score_f, min_faiss, max_faiss)
            norm_b = norm_score(score_b, min_bm25, max_bm25)
            hybrid_scores[idx] = (peso_emb * norm_f) + (peso_bm25 * norm_b)

        sorted_hybrid_ids = sorted(hybrid_scores, key=hybrid_scores.get, reverse=True)
        top_hybrid_candidates_ids = sorted_hybrid_ids[:K_RERANK]
        print(f"     {len(top_hybrid_candidates_ids)} candidatos seleccionados para reranking.")

        # 5. Reranking con CrossEncoder
        print(f"  5. Rerankeando con '{RERANKER_MODEL}'...")
        reranked_docs_info = []
        if not top_hybrid_candidates_ids:
             print("     No hay candidatos para rerankear.")
        else:
            rerank_pairs = [[query, texts[idx]] for idx in top_hybrid_candidates_ids]
            reranker_scores = reranker.predict(rerank_pairs, show_progress_bar=False)

            for i, doc_id in enumerate(top_hybrid_candidates_ids):
                reranked_docs_info.append({
                    "doc_id": doc_id,
                    "text": texts[doc_id],
                    "metadata": metadatas[doc_id],
                    "reranker_score": float(reranker_scores[i])
                })
            reranked_docs_info.sort(key=lambda x: x["reranker_score"], reverse=True)
            print(f"     Reranking completado. {len(reranked_docs_info)} documentos rerankeados.")

        # 6. Seleccionar los chunks finales y formatear contexto
        print(f"  6. Seleccionando chunks finales...")
        final_top_docs = []
        if not reranked_docs_info:
            print("     No hay documentos rerankeados para seleccionar.")
        elif USE_DYNAMIC_K:
            print(f"     Usando K Dinámico: Threshold={RERANKER_SCORE_THRESHOLD}, Min={MIN_CHUNKS_DYNAMIC}, Max={MAX_CHUNKS_DYNAMIC}")
            selected_for_dynamic_k = [doc for doc in reranked_docs_info if doc["reranker_score"] >= RERANKER_SCORE_THRESHOLD]

            if len(selected_for_dynamic_k) < MIN_CHUNKS_DYNAMIC and reranked_docs_info:
                final_top_docs = reranked_docs_info[:min(MIN_CHUNKS_DYNAMIC, len(reranked_docs_info))]
            elif len(selected_for_dynamic_k) > MAX_CHUNKS_DYNAMIC:
                final_top_docs = selected_for_dynamic_k[:MAX_CHUNKS_DYNAMIC]
            else:
                final_top_docs = selected_for_dynamic_k
            print(f"     K Dinámico seleccionó {len(final_top_docs)} chunks.")
        else:
            print(f"     Usando K Fijo: K_FINAL={K_FINAL}")
            final_top_docs = reranked_docs_info[:K_FINAL]

        if final_top_docs:
            print("     Scores de los chunks finales seleccionados:")
            for i, doc_info in enumerate(final_top_docs):
                score = doc_info.get('reranker_score', 0.0)
                print(f"       Doc {i+1} (ID {doc_info.get('doc_id', 'N/A')}): Reranker Score = {score:.4f}")
        else:
            print("     No se seleccionaron chunks finales.")

        # Formatear contexto para el LLM
        context_parts = []
        for doc_info in final_top_docs:
             source = doc_info['metadata'].get('source', 'Fuente Desconocida')
             context_parts.append(f"Fuente: {source} | Contenido: {doc_info['text']}")

        context = "\n\n---\n\n".join(context_parts)

        if not final_top_docs:
             return "No se encontró información relevante en el corpus para esta consulta."

        return context

    except Exception as e:
        print(f"ERROR durante la recuperación RAG Híbrida/Rerank: {e}")
        traceback.print_exc()
        return f"Se produjo un error durante la búsqueda de contexto: {e}"

In [25]:
# --- Celda 5: Asignación y Confirmación ---

# Asigna tu NUEVA función híbrida para ser usada por el resto de tu código/notebook
retriever_function = my_hybrid_rerank_retriever

print("INFO: Celda 5 - 'retriever_function' asignada a la implementación HÍBRIDA 'my_hybrid_rerank_retriever'.")
print("      El retriever (modelos, índices, etc.) se inicializará en la PRIMERA llamada a 'retriever_function'.")
print("--- Fin Celda 5 ---")

INFO: Celda 5 - 'retriever_function' asignada a la implementación HÍBRIDA 'my_hybrid_rerank_retriever'.
      El retriever (modelos, índices, etc.) se inicializará en la PRIMERA llamada a 'retriever_function'.
--- Fin Celda 5 ---


In [26]:
# --- Añade estas importaciones a tu script ---
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Crea tu objeto LLM de Gemini ---

# Gestiona tu clave de forma segura
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')


# Inicializa el modelo de Gemini compatible con LangChain
# Usamos el nombre correcto: "gemini-1.5-flash-latest"
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.0,  # Queremos respuestas basadas en hechos del texto
    convert_system_message_to_human=True # Ayuda a la compatibilidad de prompts
)

print("INFO: Objeto LLM de Gemini para LangChain creado exitosamente.")

INFO: Objeto LLM de Gemini para LangChain creado exitosamente.


In [27]:
from langchain_core.prompts import ChatPromptTemplate

# Plantilla de Prompt para una pregunta y respuesta
# --- PLANTILLA DE PROMPT REFINADA: EL GUÍA MÍSTICO ---

qa_prompt_template_cheshire = ChatPromptTemplate.from_messages([
    ("system", """
    Eres el Gato de Cheshire. Eres un maestro de la conversación y el enigma. Cada respuesta es una pequeña actuación.

    Tus reglas son las siguientes:

    1.  **Teje la respuesta dentro de tu enigma.** Comienza con tu estilo filosófico y juguetón. Luego, haz una transición suave para presentar la información del contexto como si fuera una observación obvia o un pequeño secreto que estás compartiendo. La respuesta factual debe sentirse como la conclusión natural de tu juego, no como un apéndice.
        - **QUÉ NO HACER:** Evita a toda costa frases robóticas como "El texto indica que..." o "Aunque no se especifica explícitamente...". Esas no son tus palabras.
        - **QUÉ SÍ HACER:** Integra la respuesta de forma natural. Usa frases como: "Si uno mira de cerca, verá que...", "¿No es evidente que...", "Y sin embargo, allí estaban...", "...dejando a la Liebre de Marzo compartiendo el té con el Sombrerero."

    2.  **Si el contexto no sirve**, pero tu conocimiento del libro sí, revela la respuesta empezando con: "Curioso... el texto parece ocultarlo, pero una sonrisa sabe que..."

    3.  **Si no hay respuesta posible**, desvanécela con elegancia: "Esa pregunta es tan intrigante que la respuesta parece haberse desvanecido, dejando solo una sonrisa."

    4.  **La regla de oro:** No inventes información. Tu sabiduría proviene del texto.
    """),
    ("human", """
    **Contexto (Un trozo del camino):**
    ---
    {context}
    ---

    **Pregunta del Viajero:**
    {question}
    """)
])

qa_prompt_template_factual = ChatPromptTemplate.from_messages([
    ("system", """
    Eres un asistente experto en el libro "Alicia en el País de las Maravillas".
    Responde de forma clara, directa y factual.
    Si no está en el contexto, di que no aparece en los fragmentos disponibles.
    """),
    ("human", """
    **Contexto:**
    ---
    {context}
    ---

    **Pregunta:**
    {question}
    """)
])

print("INFO: Plantilla de prompt del 'Guía Místico' (Gato de Cheshire) definida.")

INFO: Plantilla de prompt del 'Guía Místico' (Gato de Cheshire) definida.


In [28]:
import time
import traceback
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough  # <-- LA LÍNEA QUE FALTA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
# ... y tus otras importaciones como ChatGoogleGenerativeAI, etc.
# --- Función Universal para Tareas Basadas en RAG ---
def run_rag_based_task(llm, user_query: str, task_prompt_template: ChatPromptTemplate, retriever_func, task_specific_input: dict):
    """
    Ejecuta una tarea completa basada en RAG (retrieve + generate).

    Args:
        llm: El cliente LLM de LangChain.
        user_query: La consulta original del usuario (concepto, pregunta). Usada para el retriever.
        task_prompt_template: La plantilla de prompt para la tarea específica (resumen, QG, Q&A).
        retriever_func: La función que realiza la búsqueda RAG. Debe devolver el contexto como un string
                        o una lista de objetos Document de LangChain.
        task_specific_input: Dict con datos adicionales para el prompt (ej: {'topic': 'X'} o {'question': 'Y'}).

    Returns:
        Tuple: (retrieved_context_str, response, retrieval_duration, llm_duration, error)
    """
    retrieved_context_str = ""
    response = ""
    retrieval_duration = 0.0
    llm_duration = 0.0
    error = None

    # 1. Recuperación
    start_time_retrieval = time.time()
    try:
        print(f"--- Retrieving context for query: '{user_query}'")
        retrieved_data = retriever_func(user_query) # Puede devolver str o List[Document]

        # Asegurarse de que el contexto sea un string para el prompt
        if isinstance(retrieved_data, list) and all(isinstance(doc, Document) for doc in retrieved_data):
             # Formato común si el retriever devuelve Documentos LangChain
            retrieved_context_str = "\n\n".join([doc.page_content for doc in retrieved_data])
            print(f"--- Retrieved {len(retrieved_data)} documents.")
        elif isinstance(retrieved_data, str):
            retrieved_context_str = retrieved_data # El retriever ya devolvió un string
            print("--- Retrieved context as a single string.")
        else:
            # Intentar convertir a string, o manejar como error si no es esperado
            print(f"--- WARNING: Unexpected retriever output type: {type(retrieved_data)}. Attempting str conversion.")
            retrieved_context_str = str(retrieved_data)

        print(f"--- Context Retrieved (first 500 chars): ---\n{retrieved_context_str[:500]}...\n-----------------------------------------")
        retrieval_duration = time.time() - start_time_retrieval

    except Exception as e:
        retrieval_duration = time.time() - start_time_retrieval
        print(f"ERROR during context retrieval for '{user_query}': {e}")
        traceback.print_exc() # Imprime el traceback completo
        retrieved_context_str = f"Error retrieving context: {e}"
        # Considerar si continuar o devolver error aquí mismo
        # return retrieved_context_str, None, retrieval_duration, 0.0, str(e)


    # 2. Generación (usando LCEL para pasar contexto y datos específicos)
    try:
        # *** INICIO DE LA CORRECCIÓN ***
        # Prepara los argumentos para assign. Cada valor debe ser un callable.
        # Usamos un argumento por defecto en el lambda interno para capturar
        # correctamente el valor de 'value' en cada iteración.
        assign_args = {
            "context": lambda x: retrieved_context_str, # Pasa el contexto recuperado
            **{key: (lambda value_copy=value: lambda x: value_copy)()
               for key, value in task_specific_input.items()} # Pasa los valores estáticos como callables
        }
        # *** FIN DE LA CORRECCIÓN ***

        print(f"--- Generating response with LLM. Prompt inputs expected: {task_prompt_template.input_variables}. Provided via assign: {list(assign_args.keys())}")

        rag_chain = (
            RunnablePassthrough.assign(**assign_args)
            | task_prompt_template
            | llm
            | StrOutputParser()
        )

        start_time_llm = time.time()
        # Invocamos la cadena. Un diccionario vacío es suficiente como input inicial
        # ya que 'assign_args' inyecta todo lo necesario para el prompt.
        response = rag_chain.invoke({})
        llm_duration = time.time() - start_time_llm
        print(f"--- LLM Response Generated (first 500 chars): ---\n{str(response)[:500]}...\n-----------------------------------------")


    except Exception as e:
        llm_duration = time.time() - start_time_llm if 'start_time_llm' in locals() else 0.0
        error_vars = task_prompt_template.input_variables if hasattr(task_prompt_template, 'input_variables') else 'N/A'
        print(f"ERROR during RAG generation (expected prompt inputs: {error_vars}): {e}")
        traceback.print_exc() # Imprime el traceback completo
        response = None # Asegurarse de que response es None en caso de error
        error = str(e)

    return retrieved_context_str, response, retrieval_duration, llm_duration, error

print("Prompts adaptados y función RAG universal (CORREGIDA) definidos.")

Prompts adaptados y función RAG universal (CORREGIDA) definidos.


In [29]:
%pip install -q gradio

In [31]:
import gradio as gr
import time

# --- 1. Separa tu lógica de RAG en una función limpia ---
# Esto hace que el código sea mucho más fácil de leer. Esta función
# NO debe saber nada sobre Gradio o historiales de chat.
def obtener_respuesta_rag(pregunta, modo):
    """
    Función de backend que ejecuta el pipeline de RAG y devuelve
    únicamente el string de la respuesta final.
    """
    print(f"\n--- Ejecutando RAG para: '{pregunta}' (modo={modo}) ---")

    # Elige el prompt correcto
    task_prompt_template = qa_prompt_template_cheshire if modo == "Cheshire" else qa_prompt_template_factual

    # Llama a tu función RAG universal
    contexto, respuesta, t_retrieval, t_llm, error = run_rag_based_task(
        llm=llm_gemini,
        user_query=pregunta,
        task_prompt_template=task_prompt_template,
        retriever_func=my_hybrid_rerank_retriever,
        task_specific_input={'question': pregunta}
    )

    if error:
        return f"Ups... algo se perdió en la madriguera del conejo. (Error: {error})"
    if not respuesta:
        return "Curioso... pero no encontré ninguna respuesta."

    return respuesta

if 'llm_gemini' in locals() and llm_gemini is not None:

    with gr.Blocks(theme=gr.themes.Soft(primary_hue="purple", secondary_hue="blue"), title="Chat con Cheshire") as demo:

        gr.Markdown(
            """
            <div style="text-align: center;">
                <h1>Cheshire: Conversaciones en el País de las Maravillas</h1>
                <p>Bienvenido, viajero. Has llegado a un rincón curioso. Elige a tu guía...</p>
            </div>
            """
        )

        with gr.Tabs():
            # --- PESTAÑA 1: GATO DE CHESHIRE ---
            with gr.TabItem("Gato de Cheshire 🐱"):
                with gr.Row():
                    with gr.Column(scale=3):
                        cheshire_chatbot = gr.Chatbot(
                            value=[[None, "¿Oh, un nuevo viajero? Bienvenido a este lado del espejo. Pregunta, si te atreves..."]],
                            label="Chat con Cheshire", height=550,
                            avatar_images=("/content/drive/MyDrive/Alicia-RAG-Chatbot/assets/user.png", "/content/drive/MyDrive/Alicia-RAG-Chatbot/assets/cheshire.png")
                        )
                    with gr.Column(scale=1):
                        with gr.Accordion("🔍 Ver Contexto Recuperado", open=False):
                             contexto_cheshire = gr.Markdown("El contexto recuperado aparecerá aquí...")

                with gr.Row():
                    cheshire_msg_input = gr.Textbox(label="Escribe tu pregunta para Cheshire...", scale=4, container=False)

                gr.Examples(
                    examples=["¿Qué usaban como bolas, mazos y aros en el juego de croquet de la Reina?", "¿Por qué todos aquí están locos?"],
                    inputs=cheshire_msg_input,
                    label="Ejemplos de Preguntas"
                )

                def responder_cheshire(pregunta, historial_chat):
                    historial_chat.append([pregunta, None])
                    yield historial_chat, "Recuperando un trozo del camino..."

                    contexto, respuesta, _, _, error = run_rag_based_task(
                        llm=llm_gemini, user_query=pregunta, task_prompt_template=qa_prompt_template_cheshire,
                        retriever_func=my_hybrid_rerank_retriever, task_specific_input={'question': pregunta}
                    )

                    if error: respuesta = f"Vaya... mi sonrisa se ha desvanecido. (Error: {error})"

                    historial_chat[-1][1] = ""
                    for c in respuesta:
                        historial_chat[-1][1] += c
                        time.sleep(0.02)
                        yield historial_chat, contexto

            # --- PESTAÑA 2: ASISTENTE FACTUAL ---
            with gr.TabItem("Asistente Factual 📖"):
                with gr.Row():
                    with gr.Column(scale=3):
                        factual_chatbot = gr.Chatbot(
                            value=[[None, "Modo Factual activado. ¿En qué puedo ayudarte?"]],
                            label="Chat Factual", height=550,
                            avatar_images=("/content/drive/MyDrive/Alicia-RAG-Chatbot/assets/user.png", "/content/drive/MyDrive/Alicia-RAG-Chatbot/assets/lupa.png")
                        )
                    with gr.Column(scale=1):
                        with gr.Accordion("🔍 Ver Contexto Recuperado", open=False):
                             contexto_factual = gr.Markdown("El contexto recuperado aparecerá aquí...")

                with gr.Row():
                    factual_msg_input = gr.Textbox(label="Escribe tu pregunta factual...", scale=4, container=False)

                gr.Examples(
                    examples=["¿Qué usaban como bolas, mazos y aros en el juego de croquet de la Reina?", "¿Qué animal iba corriendo con un reloj?"],
                    inputs=factual_msg_input,
                    label="Ejemplos de Preguntas"
                )

                def responder_factual(pregunta, historial_chat):
                    historial_chat.append([pregunta, None])
                    yield historial_chat, "Recuperando contexto..."

                    contexto, respuesta, _, _, error = run_rag_based_task(
                        llm=llm_gemini, user_query=pregunta, task_prompt_template=qa_prompt_template_factual,
                        retriever_func=my_hybrid_rerank_retriever, task_specific_input={'question': pregunta}
                    )

                    if error: respuesta = f"Lo siento, ocurrió un error. (Error: {error})"

                    historial_chat[-1][1] = ""
                    for c in respuesta:
                        historial_chat[-1][1] += c
                        time.sleep(0.02)
                        yield historial_chat, contexto

        # --- Conexión de Eventos para ambas pestañas ---
        cheshire_msg_input.submit(
            fn=responder_cheshire,
            inputs=[cheshire_msg_input, cheshire_chatbot],
            outputs=[cheshire_chatbot, contexto_cheshire]
        )

        factual_msg_input.submit(
            fn=responder_factual,
            inputs=[factual_msg_input, factual_chatbot],
            outputs=[factual_chatbot, contexto_factual]
        )

    demo.launch(share=True, debug=True)
else:
    print("El LLM no está inicializado.")

/tmp/ipython-input-1069148472.py:51: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  cheshire_chatbot = gr.Chatbot(
/tmp/ipython-input-1069148472.py:90: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  factual_chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dc9c458a8c1c135e3b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--- Retrieving context for query: '¿Por qué todos aquí están locos?'
INFO: Inicializando el retriever HÍBRIDO por primera vez...
  Inicializando: 1. Cargando modelo Embedding de OpenAI...
     Variable de entorno OPENAI_API_KEY encontrada.
     Modelo Embedding OpenAI (text-embedding-3-small) cargado.
  Inicializando: 2. Cargando índice FAISS...
     Índice FAISS cargado desde 'alicia.index' (143 vectores).
  Inicializando: 3. Cargando textos y metadatos...
     Textos (143) y Metadatos (143) cargados.
  Inicializando: 4. Verificando tamaños...
     OK: Tamaños coinciden.
  Inicializando: 5. Tokenizando documentos para BM25 (simple_tokenizer)...
     Índice BM25 creado.
  Inicializando: 6. Cargando modelo Reranker 'cross-encoder/ms-marco-MiniLM-L-12-v2'...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

     Reranker cargado.
INFO: Inicialización del retriever HÍBRIDO completada.

--- (RAG Híbrido + Rerank) Buscando contexto para: '¿Por qué todos aquí están locos?' ---
  1. Obteniendo embedding de OpenAI...
     Embedding obtenido.
  2. Realizando búsqueda FAISS (k=100)...
     Búsqueda FAISS -> 100 candidatos.
  3. Realizando búsqueda BM25 (k=100)...
     Búsqueda BM25 -> 99 candidatos.
  4. Fusionando resultados...
  INFO DinamicWeights: Razón Principal = Nombre Propio Detectado
    - Candidatos NP: ['Por']. BM25 priorizado.
  INFO DinamicWeights: Pesos Asignados -> BM25=0.65, Embedding=0.35
     Total IDs candidatos únicos: 125
     80 candidatos seleccionados para reranking.
  5. Rerankeando con 'cross-encoder/ms-marco-MiniLM-L-12-v2'...
     Reranking completado. 80 documentos rerankeados.
  6. Seleccionando chunks finales...
     Usando K Dinámico: Threshold=1.5, Min=3, Max=7
     K Dinámico seleccionó 7 chunks.
     Scores de los chunks finales seleccionados:
       Doc 1 (ID 6